# Loop 2 Analysis: Pre-optimized Solutions Evaluation

The evaluator correctly identified that pre-optimized solutions with scores ~70 are available.
Let's evaluate them and understand the gap to target (68.92).

In [1]:
import numpy as np
import pandas as pd
import math
from numba import njit
import os

# Tree polygon template
@njit
def make_polygon_template():
    tw=0.15; th=0.2; bw=0.7; mw=0.4; ow=0.25
    tip=0.8; t1=0.5; t2=0.25; base=0.0; tbot=-th
    x = np.array([0,ow/2,ow/4,mw/2,mw/4,bw/2,tw/2,tw/2,-tw/2,-tw/2,-bw/2,-mw/4,-mw/2,-ow/4,-ow/2], np.float64)
    y = np.array([tip,t1,t1,t2,t2,base,base,tbot,tbot,base,base,t2,t2,t1,t1], np.float64)
    return x, y

@njit
def score_group(xs, ys, degs, tx, ty):
    """Calculate score = side^2 / n for a group of trees."""
    n = xs.size
    V = tx.size
    mnx = 1e300; mny = 1e300; mxx = -1e300; mxy = -1e300
    for i in range(n):
        r = degs[i] * math.pi / 180.0
        c = math.cos(r); s = math.sin(r)
        xi = xs[i]; yi = ys[i]
        for j in range(V):
            X = c*tx[j] - s*ty[j] + xi
            Y = s*tx[j] + c*ty[j] + yi
            if X < mnx: mnx = X
            if X > mxx: mxx = X
            if Y < mny: mny = Y
            if Y > mxy: mxy = Y
    side = max(mxx - mnx, mxy - mny)
    return side * side / n

# Warm up JIT
tx, ty = make_polygon_template()
test_xs = np.array([0.0], np.float64)
test_ys = np.array([0.0], np.float64)
test_degs = np.array([0.0], np.float64)
_ = score_group(test_xs, test_ys, test_degs, tx, ty)
print("JIT compiled")

JIT compiled


In [2]:
def strip(a):
    """Convert string values to float array."""
    return np.array([float(str(v).replace('s','')) for v in a], np.float64)

def load_submission(filepath):
    """Load submission and return dict of (xs, ys, degs) arrays per n."""
    df = pd.read_csv(filepath)
    df['N'] = df['id'].astype(str).str.split('_').str[0].astype(int)
    
    configs = {}
    for n, g in df.groupby('N'):
        xs = strip(g['x'].to_numpy())
        ys = strip(g['y'].to_numpy())
        degs = strip(g['deg'].to_numpy())
        configs[n] = (xs, ys, degs)
    return configs

def calculate_total_score(configs, tx, ty):
    """Calculate total score across all configurations."""
    total = 0.0
    for n, (xs, ys, degs) in configs.items():
        total += score_group(xs, ys, degs, tx, ty)
    return total

def get_per_n_scores(configs, tx, ty):
    """Get score contribution for each n."""
    scores = {}
    for n, (xs, ys, degs) in configs.items():
        scores[n] = score_group(xs, ys, degs, tx, ty)
    return scores

print("Helper functions defined")

Helper functions defined


In [3]:
# Evaluate all pre-optimized solutions
preopt_dir = '/home/code/preoptimized'
results = {}

for fname in os.listdir(preopt_dir):
    if fname.endswith('.csv'):
        fpath = os.path.join(preopt_dir, fname)
        try:
            configs = load_submission(fpath)
            score = calculate_total_score(configs, tx, ty)
            results[fname] = score
            print(f"{fname}: {score:.6f}")
        except Exception as e:
            print(f"{fname}: ERROR - {e}")

print(f"\nTarget: 68.922808")
print(f"Best pre-optimized: {min(results.values()):.6f}")
print(f"Gap: {min(results.values()) - 68.922808:.6f}")

ensemble.csv: 70.676102
saspav.csv: 70.676102
submission_70_926149550346.csv: 70.926150


best_public.csv: 70.926150


71.97.csv: 71.972027

Target: 68.922808
Best pre-optimized: 70.676102
Gap: 1.753294


In [4]:
# Load the best pre-optimized solution
best_file = min(results, key=results.get)
print(f"Best file: {best_file}")
best_configs = load_submission(os.path.join(preopt_dir, best_file))
best_scores = get_per_n_scores(best_configs, tx, ty)

# Analyze which N values contribute most to the score
contributions = sorted(best_scores.items(), key=lambda x: x[1], reverse=True)
print("\nTop 20 N values by score contribution:")
for n, score in contributions[:20]:
    print(f"  N={n}: {score:.6f}")

Best file: ensemble.csv

Top 20 N values by score contribution:
  N=1: 0.661250
  N=2: 0.450779
  N=3: 0.434745
  N=5: 0.416850
  N=4: 0.416545
  N=7: 0.399897
  N=6: 0.399610
  N=9: 0.387415
  N=8: 0.385407
  N=15: 0.379203
  N=10: 0.376630
  N=21: 0.376451
  N=20: 0.376057
  N=11: 0.375736
  N=22: 0.375258
  N=16: 0.374128
  N=26: 0.373997
  N=12: 0.372724
  N=13: 0.372323
  N=25: 0.372144


In [5]:
# Compare with our current best
our_configs = load_submission('/home/code/experiments/006_jit_optimizer/submission.csv')
our_score = calculate_total_score(our_configs, tx, ty)
our_scores = get_per_n_scores(our_configs, tx, ty)

print(f"Our current score: {our_score:.6f}")
print(f"Best pre-optimized: {min(results.values()):.6f}")
print(f"Difference: {our_score - min(results.values()):.6f}")

# Find where we're worse
print("\nN values where pre-optimized is better:")
for n in range(1, 201):
    diff = our_scores[n] - best_scores[n]
    if diff > 0.001:
        print(f"  N={n}: ours={our_scores[n]:.6f}, best={best_scores[n]:.6f}, diff={diff:.6f}")

Our current score: 134.997641
Best pre-optimized: 70.676102
Difference: 64.321539

N values where pre-optimized is better:
  N=2: ours=0.874338, best=0.450779, diff=0.423559
  N=3: ours=0.736690, best=0.434745, diff=0.301945
  N=4: ours=0.817895, best=0.416545, diff=0.401351
  N=5: ours=0.855754, best=0.416850, diff=0.438905
  N=6: ours=0.711438, best=0.399610, diff=0.311827
  N=7: ours=0.648462, best=0.399897, diff=0.248565
  N=8: ours=0.857622, best=0.385407, diff=0.472215
  N=9: ours=0.830015, best=0.387415, diff=0.442600
  N=10: ours=0.750297, best=0.376630, diff=0.373667
  N=11: ours=0.723669, best=0.375736, diff=0.347933
  N=12: ours=0.871010, best=0.372724, diff=0.498286
  N=13: ours=0.801524, best=0.372323, diff=0.429201
  N=14: ours=0.744272, best=0.370569, diff=0.373703
  N=15: ours=0.723901, best=0.379203, diff=0.344698
  N=16: ours=0.843564, best=0.374128, diff=0.469436
  N=17: ours=0.821763, best=0.370040, diff=0.451722
  N=18: ours=0.844360, best=0.368771, diff=0.475589
 

In [6]:
# Create ensemble: take best of our solution and pre-optimized for each N
ensemble_configs = {}
ensemble_score = 0.0

for n in range(1, 201):
    if our_scores[n] < best_scores[n]:
        ensemble_configs[n] = our_configs[n]
        ensemble_score += our_scores[n]
    else:
        ensemble_configs[n] = best_configs[n]
        ensemble_score += best_scores[n]

print(f"Ensemble score: {ensemble_score:.6f}")
print(f"Target: 68.922808")
print(f"Gap: {ensemble_score - 68.922808:.6f}")

Ensemble score: 70.676102
Target: 68.922808
Gap: 1.753294


In [ ]:
# Save ensemble as new submission
rows = []
for n in range(1, 201):
    xs, ys, degs = ensemble_configs[n]
    for i in range(n):
        rows.append({
            'id': f'{n:03d}_{i}',
            'x': f's{xs[i]:.6f}',
            'y': f's{ys[i]:.6f}',
            'deg': f's{degs[i]:.6f}'
        })

submission = pd.DataFrame(rows)
submission.to_csv('/home/code/preoptimized/ensemble_best.csv', index=False)
print(f"Ensemble saved with score: {ensemble_score:.6f}")